Tuning hyperparameters is a time-consuming and computation-expensive task. To choose the right values, we can use heuristics or an extensive grid search. Grid search is a popular method for parameter tuning in machine learning.

In [ ]:
import sys
import numpy as np

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

In [ ]:
n_classes = 5

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=False,
                                   validation_split=0.25)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('data',
                                                target_size = (150,150),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                 subset = "training")

validation_set = train_datagen.flow_from_directory('data',
                                            target_size = (150,150),
                                            batch_size = batch_size,
                                            class_mode = 'categorical',
                                            subset = "validation")

We can set the search space for different hyperparameters:

In [ ]:
space = {
            'batch_size' : hp.choice('batch_size', [32, 64, 128,
            256]), 'n_epochs' : 1000,
        }

In [ ]:
def get_callbacks(pars):
    callbacks =[EarlyStopping(monitor='val_acc', p    atience=5, verbose=2),
                ModelCheckpoint('checkpoints/{}.h5'.format(pars['batch_size']), save_best_only=True),
             TensorBoard('~/notebooks/logs-gridsearch', write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]
    return callbacks

Next, we define an objective function that we want to minimize. This function includes the network architecture, hyperparameters, results, and everything else you want to include. Make sure you include the status STATUS_OK in your return statement for successful runs:

In [ ]:
def f_nn(pars): 
    print ('Parameters: ', pars)
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape = (150, 150,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    optimizer = Adam()
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    history = model.fit(training_set, epochs=pars['n_epochs'], batch_size=pars['batch_size'], 
              validation_data= validation_set,
              verbose = 1, callbacks=get_callbacks(pars)) 
    
    best_epoch = np.argmax(history.history['val_acc'])
    best_val_acc = np.max(history.history['val_acc'])
    print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'val_acc': best_val_acc, 'best_epoch': best_epoch, 'eval_time': time.time(), 'status': STATUS_OK}

We can now initialize, start our grid search, and print the final results as follows:

In [ ]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=50, trials=trials)
print(best)